In [5]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torch

In [2]:
# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

transform = transforms.Compose(
    [
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
X_train, X_val= train_test_split(
trainset, test_size=0.2, random_state=42)

trainloader = torch.utils.data.DataLoader(X_train, batch_size=64,
                                          shuffle=True, num_workers=2)
validationloader = torch.utils.data.DataLoader(X_val, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [8]:
model_fe = models.alexnet(pretrained=True)

# We don't calculate the gradient for the actual fully-connected layer
for param in model_fe.parameters():
    param.requires_grad = False

# We get the number of input features of the fully-connected layer
num_ftrs = model_fe.classifier[6].in_features

# Adding a new fully connected linear layer
model_fe.classifier[6] = nn.Sequential (
    nn.Linear(num_ftrs, 300),
    nn.LeakyReLU(),
    nn.Linear(300, 10),
    nn.Softmax())

In [14]:
def validate(model, modelVal):
    correct = 0
    total = 0
    finalLoss = 0
    batchSize = 0 
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            image, label = data
            pred = model(image)
            outputs = model(image)
            loss = criterion(outputs, label)
            finalLoss += loss       
            batchSize = i
    return finalLoss/batchSize

In [15]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fe.parameters(), lr=0.001)

decayRate = 0.96
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        #64, 3, 11, 11
        # forward + backward + optimize
        outputs = model_fe(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        writer.add_scalar('Model1 Loss/train', loss.item(), epoch*625+i) 
        running_loss += loss.item()
        if i % 25 == 24:    # print every 25 mini-batches
            val_loss = validate(model_fe, 1)
            print('[%d, %5d] loss: %.3f %.3f validation loss --> learning rate %.9f' %
                  (epoch + 1, i + 1, running_loss / 25, val_loss, optimizer.param_groups[0]['lr']))
            writer.add_scalar('Model1 Validation Loss/train', val_loss, epoch*625+i) 
            running_loss = 0.0
    scheduler.step()

print('Finished Training')

[1,    25] loss: 1.761 1 validation loss --> learning rate 0.001000000
[1,    50] loss: 1.776 1 validation loss --> learning rate 0.001000000


KeyboardInterrupt: 

In [12]:
model2 = models.alexnet(pretrained=True)

# We don't calculate the gradient for the actual fully-connected layer
for param in model2.parameters():
    param.requires_grad = True

# We get the number of input features of the fully-connected layer
num_ftrs = model2.classifier[6].in_features

# Adding a new fully connected linear layer
model2.classifier[6] = nn.Linear(num_ftrs, 10)

625

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)

decayRate = 0.96
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        #64, 3, 11, 11
        # forward + backward + optimize
        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        writer.add_scalar('Model2 Loss/train', loss.item(), epoch*625+i) 
        running_loss += loss.item()
        if i % 25 == 24:    # print every 25 mini-batches
            val_loss = validate(model2, 1)
            print('[%d, %5d] loss: %.3f %.3f validation loss --> learning rate %.9f' %
                  (epoch + 1, i + 1, running_loss / 25, val_loss, optimizer.param_groups[0]['lr']))
            writer.add_scalar('Model2 Validation Loss/train', val_loss, epoch*625+i) 
            running_loss = 0.0
    scheduler.step()

print('Finished Training')